In [50]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import numpy as np
import time
import os
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

In [51]:
#https://stackoverflow.com/questions/23267409/how-to-implement-retry-mechanism-into-python-requests-library

import logging
import requests

from requests.adapters import HTTPAdapter, Retry

logging.basicConfig(level=logging.DEBUG)

s = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

pwd = os.getcwd()
if 'L2 TVL' in pwd:
    prepend = ''
else:
    prepend = 'L2 TVL/'

In [52]:
trailing_num_days = 7

In [53]:
#get all apps > 10 m tvl
all_api = 'https://api.llama.fi/protocols'
res = pd.DataFrame( r.get(all_api, headers=header).json() )
res = res[res['tvl'] > 100_000_000] ##greater than 10mil
print(len(res))
# print(res.columns)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.llama.fi:443
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocols HTTP/1.1" 200 1594379


356


In [54]:

protocols = res[['slug','chainTvls']]
re = res['chainTvls']
# r[1].keys()
protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: list(x.keys()) )
protocols

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_65434/1800157470.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,slug,chainTvls
0,makerdao,[Ethereum]
1,polygon-bridge-&-staking,[Polygon]
2,lido,"[Ethereum, Moonriver, Solana, Terra, Moonbeam]"
3,uniswap,"[Optimism, Ethereum, Polygon, Celo, Arbitrum]"
4,curve,"[Optimism, Harmony, Moonbeam, Ethereum, Aurora..."
...,...,...
351,sashimiswap,"[Binance-borrowed, Binance, Ethereum, Heco, Et..."
352,guard(helmet),"[Binance, Polygon]"
353,aavegotchi,"[pool2, Polygon-pool2, Ethereum, Polygon, Poly..."
354,idex,"[Ethereum, Polygon]"


In [55]:
# protocols = protocols[ protocols['slug'] == 'uniswap-v3' ]
# api_str = 'https://api.llama.fi/protocol/'
# ad = pd.DataFrame( r.get(api_str).json()['chainTvls'] ).T[['tokens']]
# ad

In [56]:
# api_str = 'https://api.llama.fi/protocol/uniswap'
# prot_req = r.get(api_str, headers=header).json()['chainTvls']
# # prot_req['Ethereum']
# prot_req

In [57]:
#get by app
api_str = 'https://api.llama.fi/protocol/'
# print(protocols)
prod = []
for index,proto in protocols.iterrows():
#     print(proto)
    prot = proto['slug']
    chains = proto['chainTvls']
    apic = api_str + prot
#     time.sleep(0.1)
    try:
        prot_req = s.get(apic, headers=header).json()['chainTvls']
#         print(apic)
        for ch in chains:
            ad = pd.json_normalize( prot_req[ch]['tokens'] )
            ad_usd = pd.json_normalize( prot_req[ch]['tokensInUsd'] )
#             ad = ad.merge(how='left')
            if not ad.empty:
                ad = pd.melt(ad,id_vars = ['date'])
                ad = ad.rename(columns={'variable':'token','value':'token_value'})
                ad_usd = pd.melt(ad_usd,id_vars = ['date'])
                ad_usd = ad_usd.rename(columns={'variable':'token','value':'usd_value'})
                ad = ad.merge(ad_usd,on=['date','token'])
                
                ad['date'] = pd.to_datetime(ad['date'], unit ='s') #convert to days
                ad['token'] = ad['token'].str.replace('tokens.','', regex=False)
                ad['protocol'] = prot
                ad['chain'] = ch
        #         ad['start_date'] = pd.to_datetime(prot[1])
                ad['date'] = ad['date'] - timedelta(days=1) #change to eod vs sod
                prod.append(ad)
    except:
        print('err')


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.llama.fi:443
DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/makerdao HTTP/1.1" 200 1003983


https://api.llama.fi/protocol/makerdao


DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/polygon-bridge-&-staking HTTP/1.1" 307 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): defillama-datasets.s3.eu-central-1.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://defillama-datasets.s3.eu-central-1.amazonaws.com:443 "GET /temp/protocol-Polygon%20Bridge%20&%20Staking.json HTTP/1.1" 200 14069119


https://api.llama.fi/protocol/polygon-bridge-&-staking


DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/lido HTTP/1.1" 200 443350


...

https://api.llama.fi/protocol/aavegotchi


DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/idex HTTP/1.1" 200 2261812


https://api.llama.fi/protocol/idex


DEBUG:urllib3.connectionpool:https://api.llama.fi:443 "GET /protocol/truefi HTTP/1.1" 200 588029


https://api.llama.fi/protocol/truefi


In [58]:
df_df_all = pd.concat(prod)
# df_df_all
print("done api")

,date,token,token_value,usd_value,protocol,chain
0,2022-05-09 00:00:00,MATIC,61211089.17387,5.060529e+07,makerdao,Ethereum
1,2022-05-10 00:00:00,MATIC,61211089.17387,5.437283e+07,makerdao,Ethereum
2,2022-05-11 00:00:00,MATIC,131231089.17387,8.719886e+07,makerdao,Ethereum
3,2022-05-12 00:00:00,MATIC,131264317.97387,7.932946e+07,makerdao,Ethereum
4,2022-05-13 00:00:00,MATIC,131264317.97387,9.167802e+07,makerdao,Ethereum
...,...,...,...,...,...,...
559,2022-08-04 00:00:00,TRU,240180051.00025,2.023925e+07,truefi,staking
560,2022-08-05 00:00:00,TRU,235743792.07384,2.043333e+07,truefi,staking
561,2022-08-06 00:00:00,TRU,235883822.467,2.002512e+07,truefi,staking
562,2022-08-07 00:00:00,TRU,235937812.2135,2.031708e+07,truefi,staking


In [59]:
# df_df[df_df['protocol']=='perpetual-protocol']
#filter down a bit so we can do trailing comp w/o doing every row
df_df = df_df_all[df_df_all['date'].dt.date >= date.today()-timedelta(days=trailing_num_days +2) ]
# display(df_df['date'].drop_duplicates())
#trailing comp
df_df['last_token_value'] = df_df.groupby(['token','protocol','chain'])['token_value'].shift(1)
#now actually filter
df_df = df_df[df_df['date'].dt.date >= date.today()-timedelta(days=trailing_num_days +1) ]
# display(df_df['date'].drop_duplicates())
# df_df[(df_df['protocol'] == 'uniswap') & (df_df['token'] != 'USDT') & (df_df['token_value'] > 0)]

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_65434/3937920575.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



83     2022-07-31 00:00:00
84     2022-08-01 00:00:00
85     2022-08-02 00:00:00
86     2022-08-03 00:00:00
87     2022-08-04 00:00:00
88     2022-08-05 00:00:00
89     2022-08-06 00:00:00
90     2022-08-07 00:00:00
91     2022-08-07 14:59:28
138    2022-08-07 15:58:33
23     2022-08-07 10:59:05
1161   2022-08-07 15:32:45
259    2022-08-07 12:59:47
384    2022-08-07 09:59:08
40     2022-08-07 08:59:28
25     2022-08-07 12:59:44
191    2022-08-07 13:59:27
95     2022-08-04 06:59:34
18     2022-08-07 15:01:46
Name: date, dtype: datetime64[ns]

,date,token,token_value,usd_value,protocol,chain,last_token_value


In [60]:
# # DISTINCT TOKENS

# token_list = df_df.groupby(['token']).count()
# token_list = token_list[token_list['token_value']>=1000]
# token_list.sort_values(by='token_value',inplace=True, ascending=False)
# token_list.reset_index(inplace=True)
# missing_token_list = token_list[~token_list['token'].isin(cg_token_list)]
# missing_token_list
# missing_token_list

In [61]:
# data_df = df_df.merge(cg_df, on=['date','token'],how='inner')
data_df = df_df.copy()
data_df['token_value'] = data_df['token_value'].replace(0, np.nan)
data_df['price_usd'] = data_df['usd_value']/data_df['token_value']

# print(len(data_df))
data_df.sort_values(by='date',inplace=True)
# print(len(data_df))
# data_df['net_token_flow'] = data_df.groupby(['token','protocol','chain'])['token_value'].apply(lambda x: x- x.shift(1))
data_df['net_token_flow'] = data_df['token_value'] - data_df['last_token_value']

# print(len(data_df))
data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']


# data_df = data_df[data_df['token'] != 'RVRS'] #Harmony error
# data_df = data_df[data_df['token'] != 'usn'] #Harmony error
# data_df = data_df[data_df['protocol'] != 'shibaswap'] #Harmony error
data_df = data_df[abs(data_df['net_dollar_flow']) < 50_000_000_000] #50 bil error bar
data_df = data_df[~data_df['net_dollar_flow'].isna()] #50 bil error bar
# display(data_df[
# #         (data_df['protocol'] == 'sushiswap') & (data_df['chain'] == 'Harmony')
#         (data_df['net_dollar_flow'].notna()) & (data_df['net_dollar_flow'] != 0)
#         ].sort_values('net_dollar_flow'))


# data_df.sort_values('net_dollar_flow')


,date,token,token_value,usd_value,protocol,chain,last_token_value,price_usd,net_token_flow,net_dollar_flow
1337,2022-08-02 00:00:00,USDT,5.948906e+09,5.960804e+09,uniswap,Ethereum,6131967304.31522,1.002000,-183061485.037889,-183427608.007965
1341,2022-08-06 00:00:00,USDT,6.174989e+09,6.174427e+09,uniswap,Ethereum,6293563981.69229,0.999909,-118574703.973881,-118563913.675819
3094,2022-08-07 14:59:28,WETH,3.999561e+05,7.192811e+08,aave-v2,Ethereum,463883.84492,1798.400000,-63927.71114,-114967595.713469
591,2022-08-04 00:00:00,USDC,5.562704e+08,5.568266e+08,aave-v3,Avalanche-borrowed,644464803.18427,1.001000,-88194448.30609,-88282642.754396
393,2022-07-31 00:00:00,USDC,6.226075e+07,6.232301e+07,truefi,borrowed,149038163.93211,1.001000,-86777410.31695,-86864187.72726
...,...,...,...,...,...,...,...,...,...,...
2678,2022-08-05 00:00:00,USDC,1.352260e+09,1.350531e+09,aave-v3,borrowed,1069565788.49479,0.998722,282693801.30422,282332518.626154
741,2022-08-05 00:00:00,USDC,8.096524e+08,8.086176e+08,aave-v3,Optimism-borrowed,498635772.33436,0.998722,311016598.83937,310619119.626053
125,2022-07-31 00:00:00,USDT,2.413396e+09,2.418223e+09,instadapp,Ethereum,2095690377.2075,1.002000,317705704.61725,318341116.026485
2677,2022-08-04 00:00:00,USDC,1.069566e+09,1.070635e+09,aave-v3,borrowed,663186989.34406,1.001000,406378799.15073,406785177.949879


In [62]:
# data_df[['date']].drop_duplicates().sort_values('date')
# data_df[data_df['date'].dt.date != data_df['date']].sort_values('date')

,date
83,2022-07-31 00:00:00
206056,2022-08-01 00:00:00
3568,2022-08-02 00:00:00
10109,2022-08-03 00:00:00
9235,2022-08-04 00:00:00
95,2022-08-04 06:59:34
1049,2022-08-05 00:00:00
25305,2022-08-06 00:00:00
154,2022-08-07 00:00:00
163,2022-08-07 08:59:28


In [63]:
# data_df[data_df['protocol']=='perpetual-protocol'].sort_values(by='date')

# segment_cols = ['date','protocol','chain','token']

In [64]:
netdf_df = data_df[['date','protocol','chain','net_dollar_flow','usd_value']]
netdf_df = netdf_df.fillna(0)
netdf_df = netdf_df.groupby(['date','protocol','chain']).sum(['net_dollar_flow','usd_value']) ##agg by app
# display(netdf_df)
#usd_value is the TVL on a given day
netdf_df = netdf_df.groupby(['date','protocol','chain','usd_value']).sum(['net_dollar_flow'])
netdf_df.reset_index(inplace=True)
# display(netdf_df)
netdf_df['cumul_net_dollar_flow'] = netdf_df[['protocol','chain','net_dollar_flow']]\
                                    .groupby(['protocol','chain']).cumsum()
netdf_df.reset_index(inplace=True)
netdf_df.drop(columns=['index'],inplace=True)


In [65]:
#get latest
netdf_df['rank_desc'] = netdf_df.groupby(['protocol', 'chain'])['date'].\
                            rank(method='dense',ascending=False).astype(int)
# netdf_df[netdf_df['chain'] == 'Klaytn']

In [71]:
summary_df = netdf_df[( netdf_df['rank_desc'] == 1 ) &\
                        (~netdf_df['chain'].str.contains('borrowed')) &\
                        (~netdf_df['chain'].str.contains('staking'))  
                        ]
summary_df = summary_df.sort_values(by='cumul_net_dollar_flow',ascending=False)
summary_df['pct_of_tvl'] = 100* summary_df['net_dollar_flow'] / summary_df['usd_value']
summary_df['flow_direction'] = np.where(summary_df['cumul_net_dollar_flow']>=0,1,-1)
summary_df['abs_cumul_net_dollar_flow'] = abs(summary_df['cumul_net_dollar_flow'])
# display(summary_df)
fig = px.treemap(summary_df[summary_df['abs_cumul_net_dollar_flow'] !=0], \
                 path=[px.Constant("all"), 'chain', 'protocol'], \
#                  path=[px.Constant("all"), 'token', 'chain', 'protocol'], \
                 values='abs_cumul_net_dollar_flow', color='flow_direction'
#                 ,color_discrete_map={'-1':'red', '1':'green'})
                ,color_continuous_scale='Spectral'
                )
# fig.data[0].textinfo = 'label+text+value'
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
# fig.update_layout(tickprefix = '$')

# fig.show()
fig.write_image(prepend + "img_outputs/svg/net_app_flows.svg") #prepend + 
fig.write_image(prepend + "img_outputs/png/net_app_flows.png") #prepend + 
fig.write_html(prepend + "img_outputs/net_app_flows.html", include_plotlyjs='cdn')

/Users/michaelsilberling/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1616: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/michaelsilberling/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1616: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/michaelsilberling/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1616: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [67]:
# netdf_df_plot = netdf_df[netdf_df['chain']=='Ethereum']

# fig = px.line(netdf_df_plot, x="date", y="net_dollar_flow", color="protocol", \
#              title="Daily Net Dollar Flow since Program Announcement",\
#             labels={
#                      "date": "Day",
#                      "net_dollar_flow": "Net Dollar Flow (N$F)"
#                  }
#             )
# fig.update_layout(
#     legend_title="App Name"
# )
# fig.update_layout(yaxis_tickprefix = '$')
# fig.show()

# # cumul_fig = px.area(netdf_df, x="date", y="cumul_net_dollar_flow", color="protocol", \
# #              title="Cumulative Dollar Flow since Program Announcement",\
# #                    labels={
# #                      "date": "Day",
# #                      "cumul_net_dollar_flow": "Cumulative Net Dollar Flow (N$F)"
# #                  }
# #             ,areamode='group')
# # cumul_fig.update_layout(yaxis_tickprefix = '$')
# # cumul_fig.show()


# cumul_fig = go.Figure()
# proto_names = netdf_df_plot['protocol'].drop_duplicates()
# print(proto_names)
# for p in proto_names:
#     cumul_fig.add_trace(go.Scatter(x=netdf_df_plot[netdf_df_plot['protocol'] == p]['date'] \
#                                    , y=netdf_df_plot[netdf_df_plot['protocol'] == p]['cumul_net_dollar_flow'] \
#                                     ,name = p\
#                                   ,fill='tozeroy')) # fill down to xaxis

# cumul_fig.update_layout(yaxis_tickprefix = '$')
# cumul_fig.update_layout(
#     title="Cumulative Dollar Flow since Program Announcement",
#     xaxis_title="Day",
#     yaxis_title="Cumulative Net Dollar Flow (N$F)",
#     legend_title="App Name",
# #     color_discrete_map=px.colors.qualitative.G10
# )
# cumul_fig.show()

In [75]:
# ! jupyter nbconvert --to python total_app_net_flows.ipynb

[NbConvertApp] Converting notebook total_app_net_flows.ipynb to python
[NbConvertApp] Writing 9870 bytes to total_app_net_flows.py
